In [433]:
import os
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
from scipy.io import loadmat
%matplotlib inline
sns.set()
sns.set_style('white')

In [434]:
data = loadmat('E:/Hamed/Amozesh/AI/Coursera-ML/16. Recommender Systems (Week 9)/ex8/ex8_movies')
r = data['R']
y = data['Y']

In [435]:
def average_rating(y, r, which_movie):
    return np.dot(y[which_movie, :], r[which_movie, :].T)

In [436]:
data = loadmat('E:/Hamed/Amozesh/AI/Coursera-ML/16. Recommender Systems (Week 9)/ex8/ex8_movieParams')
x = data['X']
theta = data['Theta']

In [437]:
num_users = 4
num_movies = 5
num_features = 3

x = x[:num_movies, :num_features]
theta = theta[:num_users, :num_features]
y = y[:num_movies, 0:num_users]
r = r[:num_movies, 0:num_users]

In [438]:
def flattenParams(myX, myTheta):
    return np.concatenate((myX.flatten(),myTheta.flatten()))

In [439]:
def reshapeParams(flattened_XandTheta, mynm, mynu, mynf):
    assert flattened_XandTheta.shape[0] == int(nm*nf+nu*nf)
    
    reX = flattened_XandTheta[:int(mynm*mynf)].reshape((mynm,mynf))
    reTheta = flattened_XandTheta[int(mynm*mynf):].reshape((mynu,mynf))
    
    return reX, reTheta

In [440]:
def collaborative_filtering_cost_function(theta, x, y, r, l):
    j = 1 / 2 * np.sum(np.square((np.dot(x, theta.T) - y) * r)) + l / 2 * (np.sum(np.square(theta)) + np.sum(np.square(x)))
    return j

In [441]:
collaborative_filtering_cost_function(theta, x, y, r, 1.5)

31.34405624427422

In [442]:
def collaborative_filtering_gradient(theta, x, y, r, l):
    Xgrad = np.dot((np.dot(x, theta.T) - y) * r, theta) + l * x
    Thetagrad = np.dot(((np.dot(x, theta.T) - y) * r).T, x) + l * theta
        
    return flattenParams(Xgrad, Thetagrad)

In [443]:
movies = []
with open('E:/Hamed/Amozesh/AI/Coursera-ML/16. Recommender Systems (Week 9)/ex8/movie_ids.txt') as f:
    for line in f:
        movies.append(' '.join(line.strip('\n').split(' ')[1:]))
my_ratings = np.zeros((1682,1))
my_ratings[0]   = 4
my_ratings[97]  = 2
my_ratings[6]   = 3
my_ratings[11]  = 5
my_ratings[53]  = 4
my_ratings[63]  = 5
my_ratings[65]  = 3
my_ratings[68]  = 5
my_ratings[182] = 4
my_ratings[225] = 5
my_ratings[354] = 5

In [444]:
mat = loadmat('E:/Hamed/Amozesh/AI/Coursera-ML/16. Recommender Systems (Week 9)/ex8/ex8_movies')
Y = mat['Y']
R = mat['R']
nf = 10

In [445]:
myR_row = my_ratings > 0
Y = np.hstack((Y,my_ratings))
R = np.hstack((R,myR_row))
nm, nu = Y.shape

In [446]:
def normalizeRatings(myY, myR):
    Ymean = np.sum(myY,axis=1)/np.sum(myR,axis=1)
    Ymean = Ymean.reshape((Ymean.shape[0],1))
    
    return myY-Ymean, Ymean

In [447]:
Ynorm, Ymean = normalizeRatings(Y,R)

In [448]:
X = np.random.rand(nm, nf)
Theta = np.random.rand(nu, nf)
myflat = flattenParams(X, Theta)
mylambda = 10.
result = scipy.optimize.fmin_cg(collaborative_filtering_cost_function, x0 = myflat, fprime = collaborative_filtering_gradient, 
                                args = (Y, R, nu, nm, nf, mylambda),
                                maxiter=50, disp=True, full_output=True)

TypeError: collaborative_filtering_gradient() takes 5 positional arguments but 7 were given

In [ ]:
resX, resTheta = reshapeParams(result[0], nm, nu, nf)
prediction_matrix = resX.dot(resTheta.T)
my_predictions = prediction_matrix[:,-1] + Ymean.flatten()

In [ ]:
pred_idxs_sorted = np.argsort(my_predictions)
pred_idxs_sorted[:] = pred_idxs_sorted[::-1]

print "Top recommendations for you:"
for i in xrange(10):
    print 'Predicting rating %0.1f for movie %s.' % \
    (my_predictions[pred_idxs_sorted[i]],movies[pred_idxs_sorted[i]])
    
print "\nOriginal ratings provided:"
for i in xrange(len(my_ratings)):
    if my_ratings[i] > 0:
        print 'Rated %d for movie %s.' % (my_ratings[i],movies[i])